P1 discontinuous SWIPDG, stationary linear elliptic ESV2007 problem
==================================

This example is about approximating the solution $u$ of the elliptic problem

$$\begin{align}
  -\nabla\cdot( \kappa \nabla u ) &= f   &&\text{in } \Omega\\
                                u &= g_D &&\text{on }\partial\Omega
\end{align}$$

with datafunction as defined in `dune/gdt/test/linearelliptic/problems/ESV2007.hh` (see below) using a piecewise linear block SWIPDG scheme, as in `dune/gdt/test/linearelliptic/discretizers/block-ipdg.hh`.

In [ ]:
import numpy as np

In [ ]:
from dune.xt.common import init_logger, init_mpi
init_mpi()
init_logger()

$$\begin{align}
  \Omega &= [-1, 1]^2\\
  \Gamma_D &= \partial\Omega\\
  \Gamma_N &= \emptyset
\end{align}$$

In [ ]:
from dune.xt.grid import (make_cube_dd_subdomains_grid__2d_simplex_aluconform as make_grid,
                          make_boundary_info_on_dd_subdomain_layer as make_boundary_info)

# the inner_boundary_segment_index must be larger than the number of boundary intersections of the fine grid
# a good guess is std::numeric_limits<size_t>::max() - 42 = 18446744073709551615 - 42 = 18446744073709551573
inner_boundary_id = 18446744073709551573
grid = make_grid(lower_left=[-1, -1], upper_right=[1, 1], num_elements=[4, 4], num_refinements=2,
                 num_partitions=[2, 2], num_oversampling_layers=0, inner_boundary_segment_index=inner_boundary_id)
grid.visualize('../block_swipdg_esv2007_grid', with_coupling=True)

boundary_info = make_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
                                          'default': 'dirichlet',
                                          'neumann': '[{} {}]'.format(inner_boundary_id, inner_boundary_id+1)})

$$\begin{align}\lambda(x) &:= 1\\
\kappa(x) &:= [1, 0; 0, 1] \in \mathbb{R}^{2\times 2}\\
f(x) &:= \tfrac{1}{2} \pi^2 \cos(\tfrac{1}{2} \pi x_0) \cos(\tfrac{1}{2} \pi x_1)\\
g_D(x) &:= 0\end{align}$$

Note that the `grid` is only provided to select the correct _type_ of function; these functions do not rely on the actual grid object.

In [ ]:
from dune.xt.functions import make_constant_function_1x1, make_constant_function_2x2, make_expression_function_1x1

lmbda = make_constant_function_1x1(grid, 1.0, name='diffusion')
kappa = make_constant_function_2x2(grid, [[1., 0.], [0., 1.]], name='diffusion')
f = make_expression_function_1x1(grid, 'x', '0.5*pi*pi*cos(0.5*pi*x[0])*cos(0.5*pi*x[1])', order=3, name='force')
g_D = make_constant_function_1x1(grid, 0.0, name='dirichlet')

In [ ]:
from dune.xt.la import (IstlRowMajorSparseMatrixDouble as Matrix,
                        IstlDenseVectorDouble as Vector)
from dune.gdt import make_block_dg_dd_subdomain_part_to_1x1_fem_p1_space as make_block_space

block_space = make_block_space(grid)

print('preparing local and coupling containers ...')
local_patterns = [block_space.local_space(ii).compute_pattern('face_and_volume')
                  for ii in np.arange(block_space.num_blocks)]
coupling_patterns = {}
for ii in np.arange(grid.num_subdomains):
    for jj in grid.neighboring_subdomains(ii):
            coupling_patterns[(ii, jj)] = block_space.compute_coupling_pattern(ii, jj, 'face')
# if we did CG locally, we would additionally need those:
#boundary_patterns = {}
#for ii in grid.boundary_subdomains():
#    boundary_patterns[ii] = block_space.compute_boundary_pattern(ii, 'face')

local_matrices = [None]*grid.num_subdomains
local_vectors = [None]*grid.num_subdomains
coupling_matrices = {}
for ii in np.arange(grid.num_subdomains):
    local_matrices[ii] = Matrix(block_space.local_space(ii).size(),
                                block_space.local_space(ii).size(),
                                local_patterns[ii])
    local_vectors[ii] = Vector(block_space.local_space(ii).size())
for ii in np.arange(grid.num_subdomains):
    for jj in grid.neighboring_subdomains(ii):
        coupling_matrices[(ii, jj)] = Matrix(block_space.local_space(ii).size(),
                                             block_space.local_space(jj).size(),
                                             coupling_patterns[(ii, jj)])
# if we did CG locally, we would additionally need boundary_matrices
# and boundary_vectors (for non-trivial boundary data)

In [ ]:
from dune.xt.grid import (make_apply_on_inner_intersections_2d_simplex_aluconformgrid_dd_subdomain_coupling_part
                          as make_apply_on_inner_intersections)

from dune.gdt import (make_elliptic_swipdg_affine_factor_matrix_operator as make_elliptic_swipdg_matrix_operator,
                      make_l2_volume_vector_functional,
                      make_local_elliptic_swipdg_affine_factor_inner_integral_operator as make_local_elliptic_swipdg_coupling_operator,
                      make_local_coupling_two_form_assembler,
                      make_system_assembler)

print('assembling local containers ...')

def assemble_local_contributions(subdomain):
    ipdg_operator = make_elliptic_swipdg_matrix_operator(lmbda, kappa, boundary_info, local_matrices[subdomain],
                                                         block_space.local_space(subdomain))
    l2_functional = make_l2_volume_vector_functional(f, local_vectors[subdomain], block_space.local_space(subdomain))
    local_assembler = make_system_assembler(block_space.local_space(subdomain))
    local_assembler.append(ipdg_operator)
    local_assembler.append(l2_functional)
    local_assembler.assemble()    
    
for ii in np.arange(grid.num_subdomains):
    assemble_local_contributions(ii)

print('assembling coupling matrices ...')

local_ipdg_coupling_operator = make_local_elliptic_swipdg_coupling_operator(lmbda, kappa)
local_coupling_assembler = make_local_coupling_two_form_assembler(local_ipdg_coupling_operator)
apply_on_inner_intersections = make_apply_on_inner_intersections()

def assemble_coupling_contributions(subdomain, neighboring_subdomain):
    coupling_assembler = block_space.coupling_assembler(subdomain, neighboring_subdomain)
    coupling_assembler.append(local_coupling_assembler,
                              local_matrices[subdomain],
                              local_matrices[neighboring_subdomain],
                              coupling_matrices[(subdomain, neighboring_subdomain)],
                              coupling_matrices[(neighboring_subdomain, subdomain)],
                              apply_on_inner_intersections) # The coupling assembler visits only coupling intersections
    # between the subdomains, those only once ond those are marked as inner. Thus we need to apply on inner intersections
    # to assemble primally.
    
for ii in np.arange(grid.num_subdomains):
    for jj in grid.neighboring_subdomains(ii):
        if ii < jj: # Assemble primally (visit each intersection only once).
            assemble_coupling_contributions(ii, jj)

In [ ]:
print('creating global container ...')

from dune.xt.la import IstlDenseVectorDouble, IstlRowMajorSparseMatrixDouble, SparsityPatternDefault

global_pattern = SparsityPatternDefault(block_space.mapper.size)
for ii in np.arange(grid.num_subdomains):
    block_space.mapper.copy_local_to_global(local_patterns[ii], ii, global_pattern)
    for jj in grid.neighboring_subdomains(ii):
        block_space.mapper.copy_local_to_global(coupling_patterns[(ii, jj)], ii, jj, global_pattern)
        block_space.mapper.copy_local_to_global(coupling_patterns[(jj, ii)], jj, ii, global_pattern)
        
system_matrix = IstlRowMajorSparseMatrixDouble(block_space.mapper.size, block_space.mapper.size, global_pattern)
rhs_vector = IstlDenseVectorDouble(block_space.mapper.size, 0.)
for ii in np.arange(grid.num_subdomains):
    block_space.mapper.copy_local_to_global(local_matrices[ii], local_patterns[ii], ii, system_matrix)
    block_space.mapper.copy_local_to_global(local_vectors[ii], ii, rhs_vector)
    for jj in grid.neighboring_subdomains(ii):
        block_space.mapper.copy_local_to_global(coupling_matrices[(ii, jj)],
                                                coupling_patterns[(ii, jj)],
                                                ii, jj, system_matrix)
        block_space.mapper.copy_local_to_global(coupling_matrices[(jj, ii)],
                                                coupling_patterns[(jj, ii)],
                                                jj, ii, system_matrix)

In [ ]:
from dune.xt.la import make_solver
from dune.gdt import make_discrete_function

print('solving linear system (of size {}x{}) ...'.format(block_space.mapper.size, block_space.mapper.size))
solution = IstlDenseVectorDouble(block_space.mapper.size, 0.0)
make_solver(system_matrix).apply(rhs_vector, solution)
make_discrete_function(block_space, solution, 'solution').visualize('../block_swipdg_esv2007_solution')